# Tutorial: CNN with Pytorch using Opp dataset

In [1]:
import sys
sys.path.append('../')
sys.path.append('/home/iwasawa/ActReco/')

In [2]:
# general
import numpy as np
import pandas as pd
from progressbar import ProgressBar

# dataset
from actreco.datasets.opportunityc import Opportunity
from actreco.sampling import sampling

# modeling
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable


In [21]:
def train_batch_generator(x_list, y_list, batch_size, l_sample, nb_iter=1, categorical=False):
    nb_sample = sum([x.shape[0] for x in x_list])
    idx_set = set(range(nb_sample))
    nb_seen_sample = 0
    for x in x_list:
        nb_seen_sample += x.shape[0]
        idx_set -= set(range(nb_seen_sample-l_sample+1, nb_seen_sample+1))
    valid_idx = list(idx_set)
    
    x = np.concatenate(x_list)
    y = np.concatenate(y_list)
    
    for i in range(nb_iter):
        start = np.random.choice(valid_idx, batch_size)
        X = sampling(x, 'clips', dtype='np', start=start, l_sample=l_sample)[..., np.newaxis].swapaxes(1, 3)
        Y = sampling(y, 'clips', dtype='np', start=start, l_sample=l_sample)
        Y = np.concatenate([Y.sum(axis=1), np.ones((batch_size, 1))], axis=-1).argmax(axis=-1)
        if categorical is not False:
            Y = np.eye(categorical)[Y]
        yield (X, Y)
        
        
def test_batch_generator(x_list, y_list, batch_size, l_sample, nb_iter=1, categorical=False, seed=0):
    
    nb_sample = sum([x.shape[0] for x in x_list])
    idx_set = set(range(nb_sample))
    nb_seen_sample = 0
    for x in x_list:
        nb_seen_sample += x.shape[0]
        idx_set -= set(range(nb_seen_sample-l_sample+1, nb_seen_sample+1))
    valid_idx = list(idx_set)
    
    x = np.concatenate(x_list)
    y = np.concatenate(y_list)
    
    for i in range(nb_iter):
        r = np.random.RandomState(seed+i)

        start = r.choice(valid_idx, batch_size)
        X = sampling(x, 'clips', dtype='np', start=start, l_sample=l_sample)[..., np.newaxis].swapaxes(1, 3)
        Y = sampling(y, 'clips', dtype='np', start=start, l_sample=l_sample)
        Y = np.concatenate([Y.sum(axis=1), np.ones((batch_size, 1))], axis=-1).argmax(axis=-1)
        if categorical is not False:
            Y = np.eye(categorical)[Y]
        yield (X, Y)

In [4]:
# dataset

In [5]:
train_set = Opportunity(userID='S1,S2,S3')
x_train_list = [x[1] for x in train_set.data_list()]
y_train_list = [x[2] for x in train_set.data_list()]

In [6]:
test_set = Opportunity(userID='S4')
x_test_list = [x[1] for x in test_set.data_list()]
y_test_list = [x[2] for x in test_set.data_list()]

In [7]:
class VanilaCNN(nn.Module):
    def __init__(self, l_sample, nb_modal, nb_in_filter, nb_filter_list, kw, nb_unit, nb_out):
        super(VanilaCNN, self).__init__()
        self.l_sample = l_sample
        self.nb_modal = nb_modal
        self.nb_conv = len(nb_filter_list)
        
        if isinstance(kw, int):
            kw = [kw] * self.nb_conv
        assert len(nb_filter_list) == len(kw)
        nb_filter_list = [nb_in_filter] + nb_filter_list
        
        self.kw = kw
        self.nb_filter_list = nb_filter_list

        convs = []
        for i in range(self.nb_conv):
            convs.append(nn.Conv2d(nb_filter_list[i], nb_filter_list[i+1], (1, kw[i])))
        self.convs = nn.ModuleList(convs)
        self.fc1 = nn.Linear(self.conv_output_shape(), nb_unit)
        self.fc2 = nn.Linear(nb_unit, nb_out)
    
    def forward(self, x):
        for conv in self.convs:
            x = F.max_pool2d(F.relu(conv(x)), (1, 2))
            dropout = nn.Dropout2d(0.5)
            # x = dropout(x)
        x = x.view(-1, self.conv_output_shape())
        x = F.dropout(self.fc1(x), 0.5)
        x = F.relu(x)
        x = self.fc2(x)
        return x
    
    def conv_output_shape(self):
        length = self.l_sample
        for i in range(self.nb_conv):
            length = int((length - self.kw[i] + 1) /2)
        return length * self.nb_filter_list[-1] * self.nb_modal

In [10]:
nb_modal_dim = 2
nb_modal = train_set.nb_modal
nb_out = train_set.nb_class

kw = 3
nb_filter_list = [50, 40, 20]
nb_in_filter = 1
drop_rate = 0.5
nb_unit = 400

batch_size = 128
l_sample = 30
interval = int(l_sample) * 0.5

nb_iter = 100
report_each = 10

model = VanilaCNN(l_sample=l_sample, nb_modal=nb_modal, nb_in_filter=1, nb_filter_list=nb_filter_list, kw=kw, nb_unit=nb_unit, nb_out=nb_out).cuda()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
criterion = nn.CrossEntropyLoss()

In [11]:
model.train()

p = ProgressBar(max_value=nb_iter)
gen = train_batch_generator(x_train_list, y_train_list, batch_size, l_sample, nb_iter=nb_iter, categorical=False)
for batch_idx, (X, y) in enumerate(gen):
    batch_idx += 1
    tX = Variable(torch.from_numpy(X.astype('float32')), requires_grad=False).cuda()
    ty = Variable(torch.LongTensor(y), requires_grad=False).cuda()
    
    optimizer.zero_grad()
    output = model(tX)
    
    loss = criterion(output, ty)
    loss.backward()
    optimizer.step()
    if batch_idx % report_each == 0:
        print("{} samples, {} batch \t Loss: {:.6f}".format(batch_idx * batch_size, batch_idx, loss.data[0]))
    p.update(batch_idx)
    

 13% (13 of 100) |###                      | Elapsed Time: 0:00:00 ETA: 0:00:05

1280 samples, 10 batch 	 Loss: 2.501356


 22% (22 of 100) |#####                    | Elapsed Time: 0:00:01 ETA: 0:00:04

2560 samples, 20 batch 	 Loss: 1.417248


 33% (33 of 100) |########                 | Elapsed Time: 0:00:01 ETA: 0:00:03

...

10240 samples, 80 batch 	 Loss: 1.534630


 94% (94 of 100) |#######################  | Elapsed Time: 0:00:04 ETA: 0:00:00

11520 samples, 90 batch 	 Loss: 1.517131


 99% (99 of 100) |######################## | Elapsed Time: 0:00:05 ETA: 0:00:00

12800 samples, 100 batch 	 Loss: 1.462462


In [12]:
from sklearn import metrics

def test(model, x_list, y_list, l_sample, nb_iter=1, batch_size=2048):
    model.eval()
    test_gen = test_batch_generator(x_list, y_list, batch_size, l_sample, nb_iter=nb_iter, categorical=False)
    y_list = []
    py_list = []
    for X, y in test_gen:
        tX = Variable(torch.from_numpy(X.astype('float32')), requires_grad=False).cuda()
        py = model(tX).max(1)[1].data.cpu().numpy().reshape(-1)
        y_list.append(y)
        py_list.append(py)
    
    y = np.concatenate(y_list)
    py = np.concatenate(py_list)
    print(metrics.accuracy_score(y, py))
    print(metrics.f1_score(y, py, average='macro'))
    print(metrics.precision_score(y, py, average=None))
    print(metrics.recall_score(y, py, average=None))

In [13]:
%prun test_batch_generator(x_test_list, y_test_list, batch_size, l_sample, nb_iter=nb_iter, categorical=False)

In [16]:
%prun test(model, x_test_list, y_test_list, l_sample, nb_iter=1, batch_size=batch_size)

0.6171875
0.0448991190679
[ 0.         0.         0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.         0.         0.
  0.6171875]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 

/home/iwasawa/.pyenv/versions/anaconda3-2.5.0/envs/env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwasawa/.pyenv/versions/anaconda3-2.5.0/envs/env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [17]:
%prun np.concatenate(x_test_list)

In [18]:
# keras model for comparison
# which is much slower than pytorch

In [19]:
from keras.models import Sequential
from keras.layers import Dense, Convolution2D, MaxPool2D, Dropout, Activation, Flatten
from keras.optimizers import SGD

model = Sequential()
model.add(Convolution2D(50, kernel_size=(1, 3), input_shape=[1, nb_modal, l_sample]))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(1, 2)))
model.add(Convolution2D(40, (1, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(1, 2)))
model.add(Convolution2D(20, (1, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(1, 2)))
model.add(Flatten())
model.add(Dense(nb_unit))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_out))
model.add(Activation('softmax'))
model.summary()

optimizer = SGD(lr=0.01, momentum=0.5)

model.compile(optimizer=optimizer, loss='categorical_crossentropy')

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN Mixed dnn version. The header is from one version, but we link with a different version (4007, 6021))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 50, 113, 28)       200       
_________________________________________________________________
activation_1 (Activation)    (None, 50, 113, 28)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 113, 14)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 40, 113, 12)       6040      
_________________________________________________________________
activation_2 (Activation)    (None, 40, 113, 12)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 40, 113, 6)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 20, 113, 4)        2420      
__________

In [23]:
p = ProgressBar(max_value=report_each)
gen = train_batch_generator(x_train_list, y_train_list, batch_size, l_sample, nb_iter=nb_iter, categorical=train_set.nb_class)
for batch_idx, (X, y) in enumerate(gen):
    batch_idx += 1
    loss = model.train_on_batch(X, y)
    if batch_idx % report_each == 0:
        print("Seen {} samples, {} batch \t Loss: {:.6f}".format(batch_idx * batch_size, batch_idx, float(loss)))
        p.update(report_each)
    else:
        p.update((batch_idx % report_each))

WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'
 10% (1 of 10) |#             | Elapsed Time: 0:00:01 ETA: 159109 days, 8:59:59

Seen 1280 samples, 10 batch 	 Loss: 2.066165


 10% (1 of 10) |#             | Elapsed Time: 0:00:03 ETA: 166697 days, 4:29:59

Seen 2560 samples, 20 batch 	 Loss: 1.473514


 10% (1 of 10) |#            | Elapsed Time: 0:00:05 ETA: 164016 days, 23:29:59

...

Seen 10240 samples, 80 batch 	 Loss: 1.229994


 10% (1 of 10) |#             | Elapsed Time: 0:00:15 ETA: 157792 days, 1:59:59

Seen 11520 samples, 90 batch 	 Loss: 1.508772


100% (10 of 10) |#########################| Elapsed Time: 0:00:17 ETA:  0:00:00

Seen 12800 samples, 100 batch 	 Loss: 1.485959
